<h1> Exploration of Upsert into the MongoDB

[This is how we can do records updates](https://stackoverflow.com/questions/45672794/updating-multiple-documents-with-specific-filters-using-pymongo)

In [2]:
import pandas as pd
from mongo import mongo_driver as db_conn
import json

DB_NAME = "reviews-db"

def update_database():

    # this single function will call two others:
    # load_new_data()
    # aggregate_new_data()
    
    conn = db_conn()
    collection = conn.get_db_collection(DB_NAME, 'coe_sp18_test')
    load_new_data(collection)


#     # load data into pandas and parse/clean
#     # TODO: implement cleaning etc
#     df = pd.read_csv("data.csv", header=None)
#     print("Found {} records.".format(len(df)))

#     # insert into db
#     records = json.loads(df.T.to_json()) #.values()

#     collections = ["COE_sp18"]
#     for collection in collections:
#         coll = conn.get_db_collection(DB_NAME, collection)
#         coll.update_many(
#            {},
#            {"$set": records },
#            upsert=True
#         )

def load_new_data(collection):
    '''
    Loads new data into the database, from the data folder. Each new .csv file is inserted into the database as a new collection. When inserting,
    the only check is to see if a collection with the name of the csv already exists in the database(i.e., if COE_Spring_2018 already exists in 
    the database, it is not inserted or modified. Else, it is inserted). Therefore, once the .csv files are placed into the data folder, do not 
    modify their contents, as the updates will not natively make it to the database.
    :inputs:
    connection: a connection to a collection within the DB
    :returns:
    connection: the same connection to the same collection of documents within the DB (the document set of the collection may be modified)
    '''
    
    # Gets the list of data documents to be checked and potentially inserted. Removes non csv files
    data_files = os.listdir('data')
    for file in data_files: 
        if file[-4:] != '.csv':
            data_files.remove(file)
      
    for data_file in data_files:
        # Reading data into python from the csv
        df = pd.read_csv(data_file, header=None)

        # load the db for the given data file into a json format
        records = json.loads(df.T.to_json()) # .values()
        
        # try to update the database with the given data file
        # can change $setonInsert to $set in the below line to automatically reenter data(i.e. if the .csv files were changed)
        result = collection.update_many({'term_and_name':data_file[:-4]},{'$setOnInsert':{data_path[:-4]:records}}, upsert=True) 
        
        # Update the user on what happened
        if result.upserted_id != None:
            print('A document for '+data_file + ' was added to the database collection.')
        else:
            print('A document for '+data_file + ' already exists in the database collection and was unmodified.')
    
    # Return the connection to the collection
    return collection
    
    # 
if __name__ == '__main__':
    # establish db connection
    conn = db_conn()
    collection = conn.get_db_collection(DB_NAME, 'coe_sp18_test')
    
    load_new_data(collection)

In [60]:
import os
data_files = os.listdir('data')
for file in data_files: 
    if file[-4:] != '.csv':
        data_files.remove(file)
data_files

['data-Copy1.csv', 'data.csv']

In [ ]:
#  establish db connection
conn = db_conn()

DB_NAME = "reviews-db"
collection_name = "none"

collection = conn.get_db_collection(DB_NAME, collection_name)

# load data into pandas and parse/clean
# TODO: implement cleaning etc
df = pd.read_csv("data.csv", header=None)
print("Found {} records.".format(len(df)))

print(df.head())

# insert into db
records = json.loads(df.T.to_json()) # .values()
# The line below 'updates' janet allens 
collection.update_many({'0':'janet allen'}, {'$set':{'1':100}},upsert=True)
records
# collection.update_many(filter={'0':{'$read':'0'}}, update={'$set':records},upsert=True)
# print("Updated collection: " + collection_name+ ' from '+ DB_NAME + ' database.')

In [77]:
# DB connection stuff 
conn=db_conn()
DB_NAME = "reviews-db"
collection_name = "test"
data_path = 'data.csv'
# These will be the paths to the csv files that contain the 
data_paths = ['data.csv', 'data-Copy1.csv']

for data_file in data_paths:
    # Reading data in
    df = pd.read_csv('data/'+data_file, header=None)
    # print("Found {} records.".format(len(df)))

#     print(df.head())

    # insert into db
    records = json.loads(df.T.to_json()) # .values()

    collection = conn.get_db_collection(DB_NAME, collection_name)

    result = collection.update_many({'term_and_name':data_file[:-4]},{'$setOnInsert':{data_path[:-4]:records}}, upsert=True) 
    if result.upserted_id != None:
        print('A document for '+data_file + ' was added to the database collection.')
    else:
        print('A document for '+data_file + ' already exists in the database collection and was unmodified.')


None
A document for data.csv already exists in the database collection and was unmodified.
5c263d1f45228972d7b2f4cf
A document for data-Copy1.csv was added to the database collection.
